## WebScrapping.

In [129]:
# Connection libraries.
import urllib.request, urllib.parse, urllib.error
import requests
from bs4 import BeautifulSoup
#from langdetect import detect
import re
import hashlib 
import time

In [72]:
# The witness class.
class TheWitness: 
    
    # Static variables.
    headers = {}
    headers["User-Agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
    social_media_names_file = "./datasets/socialmedia_names.txt"
    
    # Initialize witness features.
    def __init__(self, origin_url, destination_url, destination_hash, 
                 distance_from_root):
        
        # Where was the agent created?
        self.agent_origin = origin_url
        self.agent_destination = destination_url
        self.agent_session = requests.session()
        self.destination_hash = destination_hash
        self.distance_from_root = distance_from_root
    
    # Scrap the destination website!
    def scrap(self):
        
        # Get soup object.
        soup = self.getSoupObject()
        
        # Find all links on the page!
        result = self.find_website_links(self.agent_destination, soup)
        
        internal_website_urls = result[0]
        external_website_urls = result[1]
        urls = self.join_lists(internal_website_urls, external_website_urls)
        emails_list_1 = result[2]
        social_networks_urls = result[3]
        whatsapp_cellphones = result[4]
        images_urls_1 = result[5]
   
        # Get language of the website.
        language = ""
        
        # Get text of the website.
        text = ""
        
        # Get names that appear on the website.
        names = []
        
        # Get locations that appear on the website.
        locations = []
        
        # Get coordinates of maps that appear on the website.
        coordinates = []
        
        # Get orgazations name's that appear on the website.
        organizations = []
        
        # Get dates that appear on the website.
        dates = []
        
        # Get all phone numbers that appear on the website.
        phonenumbers = []
        
        # Get images urls that appear on the website. 
        images_urls = []
        
        # Get payment accounts that appear on the website.
        payment_accounts = []
        
        # Get emails that appear on the website.
        emails = []
        
        return [self.destination_hash, # ID of the website.
                self.agent_destination, # URL of the website.
                urls,                  # URLS found on the website.
                social_networks_urls,  # Social networks urls.
                language,              # Language of the website.
                text,                  # Text of the website.
                names,                 # Names that appeared on the website.
                locations,             # Locations mentioned on website.
                coordinates,           # Coordinates of maps that appear on the website.
                organizations,         # Organization name's.
                dates,                 # Dates on the website.
                phonenumbers,          # Phonenumbers on the website.
                images_urls,           # Images urls.
                payment_accounts,      # Payment Accounts (banks, crytpo).
                emails                 # Emails on the website.
               ]
        
    ## -----------------------
    ## Webscrapping functions.
    ## -----------------------
    
    """
    This functions makes a requests to the destination website and
    initialize a BeautifulSoup object for processing it.
    """
    def getSoupObject(self):
        r = self.agent_session.get(self.agent_destination, headers = TheWitness.headers)
        data = r.text
        return BeautifulSoup(data, "html.parser")

    """
    Find all website links and related social media and phonenumbers 
    associated with urls on the page.
    Input: 
    Output:
    """
    
    # Find all the urls in the website.
    def find_website_links(self, url, soup):
        
        # Get social media names.
        f = open(TheWitness.social_media_names_file, "r")
        sm_keywords = f.read().replace("\n", "").split(" ")
        f.close()

        # Remove last / in url if exists.
        if(url[-1] == "/"):
            url = url[0: -1]

        # Initialize lists.
        internal_website_urls = list()
        external_website_urls = list()
        social_networks_urls = list()
        whatsapp_cellphones = list()
        images_urls = list()
        emails_list = list()

        for tag in soup.find_all("a"):

            try:
                # Find href attribute.
                link = tag.attrs["href"]


                # If link is not an images or #
                if link.find("#") == -1 and link.find("jpg") == -1 and link.find("png") == -1: 

                    # Determine if link corresponds to an internal navegation page.
                    if link.find("/") == 0: 
                        new_link = url + link
                        if new_link not in internal_website_urls: 
                            internal_website_urls.append(new_link)

                    # External link.
                    else: 

                        # Check if the external link is a social network link!
                        social_network_website = False
                        for social_network_name in sm_keywords: 
                            if link.find(social_network_name) != -1: 
                                social_network_website = True
                                break

                        if social_network_website:

                            if (link not in social_networks_urls): 

                                # Add link to social networks list.
                                social_networks_urls.append(link)


                                # If link is of the form api.whatsapp.com, get the phonenumber.
                                if link.find("whatsapp") != -1:

                                    number = re.findall("phone=([0-9]*)", link)
                                    if number[0] not in whatsapp_cellphones: 
                                        whatsapp_cellphones.append(number[0])     

                        # If link is not a social network link, just add it to website_urls list.
                        else: 

                            # We have an email!
                            if link.find("mailto:") != -1:
                                if link not in emails_list: 
                                    emails_list.append(link)
                            elif link not in external_website_urls: 
                                external_website_urls.append(link)

                # If the link is an image.
                elif link.find("jpg") != -1 or link.find("png") != -1:
                    if link not in images_urls: 
                        images_urls.append(link)

            except: 
                pass

        return internal_website_urls, external_website_urls, emails_list, social_networks_urls, whatsapp_cellphones, images_urls

    ## -------------------
    ## Auxiliar functions.
    ## -------------------

    """
    This method joins to python's lists.
    """   
    def join_lists(self, list1, list2): 
        new_list = list()
        for element in list1: 
            new_list.append(element)
        for element in list2: 
            new_list.append(element)
        return new_list

In [73]:
agent = TheWitness("https://dentalgomez.co/", "https://dentalgomez.co/", destination_hash = "srgh2314", distance_from_root = 1)

In [74]:
for element in agent.scrap(): 
    print(element)

srgh2314
https://dentalgomez.co/
['https://dentalgomez.co/', 'https://dentalgomez.co/contactus', 'https://dentalgomez.co/shop', 'https://dentalgomez.co/livechat', 'https://dentalgomez.co/shop/cart', 'https://dentalgomez.co/web/login', 'https://dentalgomez.co/aboutus', 'http://www.odoo.com?utm_source=db&utm_medium=sitio web', 'http://www.odoo.com/page/e-commerce?utm_source=db&utm_medium=website']
['https://www.facebook.com/dentalgomez.co', 'https://www.instagram.com/dentalgomez.co/', 'https://api.whatsapp.com/send?phone=573058270682&text=%C2%A1Hola!%20Me%20gustar%C3%ADa%20cotizar%20un%20equipo%20con%20Dental%20G%C3%B3mez.', 'https://api.whatsapp.com/send?phone=573058270682&text=%C2%A1Hola!%20Me%20gustar%C3%ADa%20contactarme%20con%20Dental%20G%C3%B3mez.']


[]
[]
[]
[]
[]
[]
[]
[]
[]


In [103]:
# Omit pages.
omit_pages = ["google", "linkto", "odoo"]

In [120]:
# Global variables.
hashes_map = {}

In [113]:
root = "https://modelosalacarta.com/"
destination = "https://modelosalacarta.com/"
hash_destination = nodeID(destination)
distance_from_root = 0
hashes_map[destination] = True

witness = TheWitness(
    root, 
    destination, 
    hash_destination, 
    distance_from_root
)
result = witness.scrap()

In [126]:
result

['3a19f99861150b1f7cb69eff29e26ac24bddf208b971f8b47571334e022a8e48',
 'https://modelosalacarta.com/',
 ['https://modelosalacarta.com/cdn-cgi/l/email-protection',
  'https://modelosalacarta.com/',
  'https://modelosalacarta.com/chicas-escorts-y-prepagos-en-bogota-colombia/',
  'https://modelosalacarta.com/chicas-escorts-y-prepagos-en-medellin/',
  'https://modelosalacarta.com/chicas-escorts-y-prepagos-en-cali/',
  'https://modelosalacarta.com/blog/',
  'https://modelosalacarta.com/agencia-de-acompanantes/',
  'https://modelosalacarta.com/pedir-una-escort-de-lujo-cali-bogota-medellin/',
  'https://modelosalacarta.com/trabajar-como-modelo-acompanante/',
  'https://modelosalacarta.com/contactenos/',
  'https://www.adult-works.com/london-adultwork/',
  'https://massagerepublic.com/female-escorts-in-bogota/sofy-purple',
  'https://massagerepublic.com/female-escorts-in-bogota',
  'https://massagerepublic.com/female-escorts-in-london',
  'https://www.eurogirlsescort.com/',
  'https://www.world

In [132]:
# For link in links.
def explore_network(result):
    
    count = 1
    for link in result[2]: 

        
        # We already explored this website.
        try: 
            if hashes_map[link]: 
                pass

        # New website!
        except:
            
            navegable = True
            for website in omit_pages: 
                if link.find(website) != -1: 
                    navegable = False
                    

            if navegable: 
                root = "https://modelosalacarta.com/"
                destination = link
                hash_destination = nodeID(destination)
                distance_from_root = 1
                #hashes_map[destination] = True

                witness = TheWitness(
                    root, 
                    destination, 
                    hash_destination, 
                    distance_from_root
                )
                result = witness.scrap()
                
                print("---------------")
                print("Agent No.", count)
                print(result)
                print("---------------")
                
                del witness
                
        count += 1

In [133]:
explore_network(result)

---------------
Agent No. 1
['15935b92cdfb04569c357c8a013a93e30fbaa38d1777ba2a1f6912cc4105a8e6', 'https://modelosalacarta.com/cdn-cgi/l/email-protection', ['https://www.cloudflare.com/sign-up?utm_source=email_protection', 'https://support.cloudflare.com/hc/en-us/articles/200170016-What-is-Email-Address-Obfuscation-', 'https://support.cloudflare.com/hc/en-us/categories/200275218-Getting-Started', 'https://www.cloudflare.com/5xx-error-landing?utm_source=error_footer'], [], '', '', [], [], [], [], [], [], [], [], []]
---------------
---------------
Agent No. 2
['3a19f99861150b1f7cb69eff29e26ac24bddf208b971f8b47571334e022a8e48', 'https://modelosalacarta.com/', ['https://modelosalacarta.com/cdn-cgi/l/email-protection', 'https://modelosalacarta.com/', 'https://modelosalacarta.com/chicas-escorts-y-prepagos-en-bogota-colombia/', 'https://modelosalacarta.com/chicas-escorts-y-prepagos-en-medellin/', 'https://modelosalacarta.com/chicas-escorts-y-prepagos-en-cali/', 'https://modelosalacarta.com/bl

---------------
Agent No. 7
['7f756a1bd0a3cf5e80ef95f732f3cb4cef54da5dba569da64f15901175ffa9e5', 'https://modelosalacarta.com/agencia-de-acompanantes/', ['https://modelosalacarta.com/agencia-de-acompanantes/cdn-cgi/l/email-protection', 'https://modelosalacarta.com/', 'https://modelosalacarta.com/chicas-escorts-y-prepagos-en-bogota-colombia/', 'https://modelosalacarta.com/chicas-escorts-y-prepagos-en-medellin/', 'https://modelosalacarta.com/chicas-escorts-y-prepagos-en-cali/', 'https://modelosalacarta.com/blog/', 'https://modelosalacarta.com/agencia-de-acompanantes/', 'https://modelosalacarta.com/pedir-una-escort-de-lujo-cali-bogota-medellin/', 'https://modelosalacarta.com/trabajar-como-modelo-acompanante/', 'https://modelosalacarta.com/contactenos/', 'https://www.google.com.co/?gws_rd=ssl'], ['https://api.whatsapp.com/send?phone=573203226262&text=Hola,%20me%20gustaria%20conocer%20el%20men%C3%BC.', 'https://twitter.com/modelosalacarta', 'https://www.instagram.com/modelos_a_la_carta/', '

---------------
Agent No. 12
['c483f8e6b751e03007b69789bdc6e4b8452608a475b28b150fabaf3c1599ae4a', 'https://massagerepublic.com/female-escorts-in-bogota/sofy-purple', ['https://massagerepublic.com/female-escorts-in-bogota/sofy-purple/sign-in', 'https://massagerepublic.com/female-escorts-in-bogota/sofy-purple/', 'https://massagerepublic.com/female-escorts-in-bogota/sofy-purple/female-escorts-in-bogota', 'https://massagerepublic.com/female-escorts-in-bogota/sofy-purple/female-escorts-in-bogota/camila-will-make-your-dreams-come-true', 'https://massagerepublic.com/female-escorts-in-bogota/sofy-purple/action/listings/sofy-purple/favourite', 'https://massagerepublic.com/female-escorts-in-bogota/sofy-purple/female-escorts-in-bogota/sofy-purple', 'https://massagerepublic.com/female-escorts-in-bogota/sofy-purple/action/listings/sofy-purple/show_phone', 'https://massagerepublic.com/female-escorts-in-bogota/sofy-purple/action/listings/sofy-purple/listing_messages/new', 'https://massagerepublic.com

---------------
Agent No. 14
['2b2d7eeeaf095fe18faa97d85b096b90cc62e7189724eebd3cdefe1c02e60f0a', 'https://massagerepublic.com/female-escorts-in-london', ['https://massagerepublic.com/female-escorts-in-london/sign-in', 'https://massagerepublic.com/female-escorts-in-london/', 'https://massagerepublic.com/female-escorts-in-london/female-escorts-in-london', 'https://massagerepublic.com/female-escorts-in-london/female-escort-news-in-london', 'https://massagerepublic.com/female-escorts-in-london/search-female-escorts', 'https://massagerepublic.com/female-escorts-in-london/male-escorts-in-london', 'https://massagerepublic.com/female-escorts-in-london/shemale-escorts-in-london', 'https://massagerepublic.com/female-escorts-in-london/massage-female-escorts-in-london', 'https://massagerepublic.com/female-escorts-in-london/gfe-female-escorts-in-london', 'https://massagerepublic.com/female-escorts-in-london/french-kissing-female-escorts-in-london', 'https://massagerepublic.com/female-escorts-in-lo

---------------
Agent No. 15
['6d9d8b7b09be9a70b7fff350558ffb17640e5bc85f747f01780de70384d75714', 'https://www.eurogirlsescort.com/', ['https://www.eurogirlsescort.com/user/over18?backlink=2ym06', 'https://www.eurogirlsescort.com/user/create-account/?type=user', 'https://www.eurogirlsescort.com/', 'https://www.eurogirlsescort.com/escorts/albania/', 'https://www.eurogirlsescort.com/escorts/durres/', 'https://www.eurogirlsescort.com/escorts/fier/', 'https://www.eurogirlsescort.com/escorts/tirana/', 'https://www.eurogirlsescort.com/escorts/armenia/', 'https://www.eurogirlsescort.com/escorts/yerevan/', 'https://www.eurogirlsescort.com/escorts/austria/', 'https://www.eurogirlsescort.com/escorts/amstetten/', 'https://www.eurogirlsescort.com/escorts/baden/', 'https://www.eurogirlsescort.com/escorts/berg/', 'https://www.eurogirlsescort.com/escorts/braunau-am-inn/', 'https://www.eurogirlsescort.com/escorts/bregenz/', 'https://www.eurogirlsescort.com/escorts/donnerskirchen/', 'https://www.eurogi

---------------
Agent No. 16
['453c09aab84dc71221e5d426459e0bba548843c34b4d222ea49818c07c95fa75', 'https://www.worldescortindex.com/', ['https://www.worldescortindex.com/', 'https://www.worldescortindex.com/our-banners/', 'https://www.worldescortindex.com/advertise/', 'https://www.worldescortindex.com/advertise/create-site/', 'https://www.worldescortindex.com/contact/', 'https://www.worldescortindex.com/escorts/albania/', 'https://www.worldescortindex.com/escorts/armenia/', 'https://www.worldescortindex.com/escorts/austria/', 'https://www.worldescortindex.com/escorts/belarus/', 'https://www.worldescortindex.com/escorts/belgium/', 'https://www.worldescortindex.com/escorts/bulgaria/', 'https://www.worldescortindex.com/escorts/croatia/', 'https://www.worldescortindex.com/escorts/cyprus/', 'https://www.worldescortindex.com/escorts/czech-republic/', 'https://www.worldescortindex.com/escorts/denmark/', 'https://www.worldescortindex.com/escorts/estonia/', 'https://www.worldescortindex.com/esc

---------------
Agent No. 17
['416826ee646dce0714c969a79035c172d05d7692bee83c790fa555d36b8a628d', 'https://escortdirectory.tv', ['https://escortdirectory.tv/cdn-cgi/l/email-protection', 'https://escortdirectory.tv', "javascript:alert('Press ' + (navigator.userAgent.toLowerCase().indexOf('mac') != - 1 ? 'Command/Cmd' : 'CTRL') + ' + D to bookmark this page.');", 'https://escortdirectory.tv/links', 'https://escortdirectory.tv/submit', 'https://escortdirectory.tv/banner', 'https://escortdirectory.tv/contact', 'https://escortdirectory.tv/escort-albania', 'https://escortdirectory.tv/escort-angola', 'https://escortdirectory.tv/escort-argentina', 'https://escortdirectory.tv/escort-armenia', 'https://escortdirectory.tv/escort-australia', 'https://escortdirectory.tv/escort-austria', 'https://escortdirectory.tv/escort-azerbaijan', 'https://escortdirectory.tv/escort-bahrain', 'https://escortdirectory.tv/escort-bangladesh', 'https://escortdirectory.tv/escort-barbados', 'https://escortdirectory.tv/

---------------
Agent No. 18
['0cc89f40b5300d6667466070f6b46c87feb1eb7391a36e9f510ea76fbad85409', 'https://topescortbabes.com', ['https://topescortbabes.com/terms', 'https://topescortbabes.com/privacy-policy', 'https://topescortbabes.com//www.dmca.com/Protection/Status.aspx?ID=e629619e-0751-4fb8-9150-4d5449b989f3', 'https://topescortbabes.com/colombia/escorts', 'https://topescortbabes.com/escorts', 'https://topescortbabes.com/pornstar-escorts', 'https://topescortbabes.com/shemale-escorts', 'https://topescortbabes.com/videos', 'https://topescortbabes.com/escort-reviews', 'https://topescortbabes.com/top-escorts', 'https://topescortbabes.com/escort-agencies', 'http://www.rtalabel.org', 'https://topescortbabes.com/usa/escorts', 'https://topescortbabes.com/los-angeles/escorts', 'https://topescortbabes.com/new-york/escorts', 'https://topescortbabes.com/france/escorts', 'https://topescortbabes.com/paris/escorts', 'https://topescortbabes.com/lyon/escorts', 'https://topescortbabes.com/uk/escort

---------------
Agent No. 21
['787b4b875cebf45780d2ed2934fb7ad5e8c3ef3a63559cfbc459cd1fe1029269', 'http://www.escortnews.com', ['http://www.escortnews.com//www.dmca.com/Protection/Status.aspx?ID=b9bd9e2d-8a33-47b4-b6d5-8c65f5b4a11f', 'https://www.escortnews.com/allcountries', 'https://albania.escortnews.com/en', 'https://angola.escortnews.com/en', 'https://argentina.escortnews.com/en', 'https://armenia.escortnews.com/en', 'https://australia.escortnews.com/en', 'https://austria.escortnews.com/en', 'https://azerbaijan.escortnews.com/en', 'https://bahrain.escortnews.com/en', 'https://bangladesh.escortnews.com/en', 'https://belarus.escortnews.com/en', 'https://belgium.escortnews.com/en', 'https://bolivia.escortnews.com/en', 'https://bosnia-and-herzegovina.escortnews.com/en', 'https://brazil.escortnews.com/en', 'https://brunei.escortnews.com/en', 'https://bulgaria.escortnews.com/en', 'https://cambodia.escortnews.com/en', 'https://canada.escortnews.com/en', 'https://chile.escortnews.com/en',

---------------
Agent No. 23
['be36efd220ea3b1f75950324dd359b5165ff0a2cc62468409a6d541f31684f83', 'https://www.escort-guide.net/', ['https://www.escort-guide.net/fr/', 'https://www.escort-guide.net/de/', 'https://www.escort-guide.net/it/', 'https://www.escort-guide.net/pt/', 'https://www.escort-guide.net/es/', 'https://www.escort-guide.net/remind-password.html', 'https://www.escort-guide.net/escort-registration.html', 'https://www.escort-guide.net/', 'https://www.escort-guide.net/call-girls.html', 'https://www.escort-guide.net/search.html', 'https://www.escort-guide.net/contact.html', 'https://www.escort-guide.net/add-escort.html', 'https://www.escort-guide.net/call-girls/advanced-search.html', 'https://fantasyescortsbirmingham.co.uk', 'https://www.torchemada.net', 'https://www.babylongirls.co.uk/london-escorts-gallery/', 'https://www.ubergirls.me', 'https://www.escort-guide.net/call-girls-british-escorts/', 'https://www.escort-guide.net/call-girls-france/', 'https://www.escort-guide.n

In [88]:
result = witness.scrap()

In [110]:
result[2]

['https://modelosalacarta.com/cdn-cgi/l/email-protection',
 'https://modelosalacarta.com/',
 'https://modelosalacarta.com/chicas-escorts-y-prepagos-en-bogota-colombia/',
 'https://modelosalacarta.com/chicas-escorts-y-prepagos-en-medellin/',
 'https://modelosalacarta.com/chicas-escorts-y-prepagos-en-cali/',
 'https://modelosalacarta.com/blog/',
 'https://modelosalacarta.com/agencia-de-acompanantes/',
 'https://modelosalacarta.com/pedir-una-escort-de-lujo-cali-bogota-medellin/',
 'https://modelosalacarta.com/trabajar-como-modelo-acompanante/',
 'https://modelosalacarta.com/contactenos/',
 'https://www.adult-works.com/london-adultwork/',
 'https://massagerepublic.com/female-escorts-in-bogota/sofy-purple',
 'https://massagerepublic.com/female-escorts-in-bogota',
 'https://massagerepublic.com/female-escorts-in-london',
 'https://www.eurogirlsescort.com/',
 'https://www.worldescortindex.com/',
 'https://escortdirectory.tv',
 'https://topescortbabes.com',
 'https://linkto.directory/',
 'http:

In [2]:
#url = "https://co.mileroticos.com"
url = "https://dentalgomez.co/"
r = requests.get(url)
data = r.text
soup = BeautifulSoup(data, "html.parser")

In [76]:
def nodeID(url): 
    m = hashlib.sha256()
    m.update(url.encode())
    return m.hexdigest()

In [24]:
# Find all the urls in the website.
def find_links(url, soup):
    
    # Get social media names.
    f = open("./datasets/socialmedia_names.txt", "r")
    sm_keywords = f.read().replace("\n", "").split(" ")
    f.close()

    # Remove last / in url if exists.
    if(url[-1] == "/"):
        url = url[0: -1]

    # Initialize lists.
    internal_website_urls = list()
    external_website_urls = list()
    social_networks_urls = list()
    whatsapp_cellphones = list()
    images_urls = list()
    emails_list = list()

    for tag in soup.find_all("a"):

        try:
            # Find href attribute.
            link = tag.attrs["href"]
            

            # If link is not an images or #
            if link.find("#") == -1 and link.find("jpg") == -1 and link.find("png") == -1: 

                # Determine if link corresponds to an internal navegation page.
                if link.find("/") == 0: 
                    new_link = url + link
                    if new_link not in internal_website_urls: 
                        internal_website_urls.append(new_link)

                # External link.
                else: 

                    print("External link: ", link)
                    
                    # Check if the external link is a social network link!
                    social_network_website = False
                    for social_network_name in sm_keywords: 
                        if link.find(social_network_name) != -1: 
                            social_network_website = True
                            break

                    if social_network_website:
                    
                        if (link not in social_networks_urls): 

                            # Add link to social networks list.
                            social_networks_urls.append(link)


                            # If link is of the form api.whatsapp.com, get the phonenumber.
                            if link.find("whatsapp") != -1:

                                number = re.findall("phone=([0-9]*)", link)
                                if number[0] not in whatsapp_cellphones: 
                                    whatsapp_cellphones.append(number[0])     

                    # If link is not a social network link, just add it to website_urls list.
                    else: 
                        
                        # We have an email!
                        if link.find("mailto:") != -1:
                            if link not in emails_list: 
                                emails_list.append(link)
                        elif link not in external_website_urls: 
                            external_website_urls.append(link)
            
            # If the link is an image.
            elif link.find("jpg") != -1 or link.find("png") != -1:
                if link not in images_urls: 
                    images_urls.append(link)
                
        except: 
            pass

    return internal_website_urls, external_website_urls, emails_list, social_networks_urls, whatsapp_cellphones, images_urls

In [27]:
a = find_links(url, soup)
a[1]

External link:  mailto:ventas@dentalgomez.co
External link:  https://www.facebook.com/dentalgomez.co
External link:  https://www.instagram.com/dentalgomez.co/
External link:  https://api.whatsapp.com/send?phone=573058270682&text=%C2%A1Hola!%20Me%20gustar%C3%ADa%20cotizar%20un%20equipo%20con%20Dental%20G%C3%B3mez.
External link:  https://api.whatsapp.com/send?phone=573058270682&text=%C2%A1Hola!%20Me%20gustar%C3%ADa%20contactarme%20con%20Dental%20G%C3%B3mez.
External link:  https://www.facebook.com/dentalgomez.co
External link:  https://www.instagram.com/dentalgomez.co/
External link:  mailto:ventas@dentalgomez.co?subject=Contacto%20con%20Dental%20Gomez&body=%C2%A1Hola!%20Deseo%20averiguar%20informaci%C3%B3n%20sobre
External link:  https://www.facebook.com/dentalgomez.co
External link:  https://www.instagram.com/dentalgomez.co/
External link:  http://www.odoo.com?utm_source=db&utm_medium=sitio web
External link:  http://www.odoo.com/page/e-commerce?utm_source=db&utm_medium=website


['http://www.odoo.com?utm_source=db&utm_medium=sitio web',
 'http://www.odoo.com/page/e-commerce?utm_source=db&utm_medium=website']

In [ ]:
url

In [ ]:
nodeID(url)

In [ ]:
internal_links, external_links = internal_external_links(url, urls)

In [ ]:
internal_links

In [ ]:
external_links

In [ ]:
social_networks

In [ ]:
wa_cellphones